# Using MLRUN function locally, as a Kubernetes Job, and in a Workflow
The following notebook demonstrates how to write and test code in a notebook, convert it to a containerized image<br>
run it on a Kubernetes cluster with shared file or object storage, and finally run it in an automated workflow

<b> Content </b><a id=top></a>
1. [Define a new function and its dependencies](#define-function)
2. [Test the function code and pipeline locally](#test-locally)
3. [Define cluster jobs and build images](#build)
4. [Run the function on the cluster](#run-on-cluster)
5. [Create and Run a KubeFlow Pipeline](#create-pipeline)

In [ ]:
# Uncomment this to install mlrun package, restart the kernel after
#!pip install mlrun

In [1]:
# nuclio: ignore
# do not remove the comment above (it is a directive to nuclio, ignore that cell during build)
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter and restart the kernel 
import nuclio 

<a id=define-function></a>
## Define a new function and its dependencies 
We use `%nuclio` magic commands to set package dependencies and configuration

In [2]:
%nuclio cmd -c pip install pandas
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Function code - example pipeline

In [3]:
import os

def training(context, p1=1, p2=2):
    # access input metadata, values, and inputs
    print(f'Run: {context.name} (uid={context.uid})')
    print(f'Params: p1={p1}, p2={p2}')
    context.logger.info('started training')
    
    # do some training 
    
    # log the run results (scalar values)
    context.log_result('accuracy', p1 * 2)
    context.log_result('loss', p1 * 3)
    
    # add a lable/tag to this run 
    context.set_label('category', 'tests')
    
    # log a simple artifact + label the artifact 
    # If you want to upload a local file to the artifact repo add src_path=<local-path>
    context.log_artifact('model', body=b'abc is 123', target_path='model.txt', 
                          labels={'framework': 'xgboost'})

    
def validation(context, model):
    # access input metadata, values, files, and secrets (passwords)
    print(f'Run: {context.name} (uid={context.uid})')
    #model = context.get_object('model', model)
    print('file - {}:\n{}\n'.format(model.url, model.get()))
    
    context.logger.info('started validation')    
    context.log_artifact('validation', body=b'<b> validated </b>', 
                         target_path='validation.html', viewer='web-app')

In [4]:
# nuclio: end-code
# (end-code marker tells nuclio to stop parsing the notebook from this cell, DO NOT REMOVE!)

<a id=test-locally></a>
## Test the function code and pipeline locally
the functions above can be tested locally, parameters, inputs, and outputs can be specified in the API or the `Task` object<br>
we create a `function` which defines the runtime environment (type, code, image, ..) and `run` a job/experiments using that function <br>
(we use the `local` runtime by default, later on we will use a `job` runtime for running containers, and can use other distributed runners like MpiJob, Spark, Dask, Nuclio, ..)

in each run we can specify the function, inputs, parameters/hyper-parameters, etc. (check the `RunTemplate` class for details)<br>
see the [mlrun_basics](mlrun_basics.ipynb) notebook for more details.

### Load MLRUN and specify defaults 

In [ ]:
# set mlrun db/api path (can also be specified in mlrun.mlconf)
%env MLRUN_DBPATH=http://<mlrun-api-url>:8080
        
# set the UI external URL (will generate ui hyperlinks)
%env MLRUN_UI_URL=http://<mlrun-ui-url>:<port>
        
from mlrun import new_function, code_to_function, NewTask, mount_v3io

### Running and linking multiple tasks
in the next example we run two functions, `training` and `validation` and we pass the result from one to the other.<br>
we will see in the 'job' example that linking works even when the tasks run on different processes or containers, or in a workflow.

In [6]:
# run the training function (new_function() will create a local function object)
# functions can have multiple handlers/methods, we call the training() handler 
fn = new_function()
train_run = fn.run(handler=training, params={'p1': 5})

Run: training (uid=db2bec7021324c7f8d1fc48e745dfa91)
Params: p1=5, p2=2
[mlrun] 2019-12-18 21:29:50,814 started training

Run: training (uid=db2bec7021324c7f8d1fc48e745dfa91)
Params: p1=5, p2=2
[mlrun] 2019-12-18 21:29:50,814 started training



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...5dfa91,0,Dec 18 21:29:50,completed,training,category=testshost=jupyter-dulwoc9x63-ixir3-68dccc6b7-rr8cn,,p1=5,accuracy=10loss=15,model


to track results use .show() or .logs() or in CLI: 
!mlrun get run db2bec7021324c7f8d1fc48e745dfa91  , !mlrun logs db2bec7021324c7f8d1fc48e745dfa91 
[mlrun] 2019-12-18 21:29:50,881 run executed, status=completed


after the function run it generates the result widget, you can click the `model` artifact to see its content

<b>The output from the first function is passed to the 2nd (validation) function</b>

In [7]:
# run the validation function
model_path = train_run.outputs['model']
validation_run = fn.run(handler=validation, inputs={'model': model_path})

Run: validation (uid=e4f7a1cd7d6147b3b5f56ae88fac1dbc)
file - model.txt:
b'abc is 123'

[mlrun] 2019-12-18 21:29:53,606 started validation

Run: validation (uid=e4f7a1cd7d6147b3b5f56ae88fac1dbc)
file - model.txt:
b'abc is 123'

[mlrun] 2019-12-18 21:29:53,606 started validation



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...ac1dbc,0,Dec 18 21:29:53,completed,validation,host=jupyter-dulwoc9x63-ixir3-68dccc6b7-rr8cn,model,,,validation


to track results use .show() or .logs() or in CLI: 
!mlrun get run e4f7a1cd7d6147b3b5f56ae88fac1dbc  , !mlrun logs e4f7a1cd7d6147b3b5f56ae88fac1dbc 
[mlrun] 2019-12-18 21:29:53,669 run executed, status=completed


<a id=build></a>
## Define cluster jobs and build images 
in order to use our function in a cluster we need to package our code and dependencies, the `code_to_function` call<br>
 will automatically form a `function` object from the current notebook (or a specified file) with list of dependencies and runtime configuration<br>

In [8]:
# create an ML function from the notebook, attache it to iguazio data fabric (v3io)
trainer = code_to_function(name='my-trainer', runtime='job')

### Define shared storage for the functions
The functions need shared storage (file or object) media to pass and store artifacts<br>
you can add `Kubernetes` resources like volumes, environment variables, secrets, cpu/mem/gpu, etc. to a function<br>
`MLRun` uses `KubeFlow` modifiers (apply) to configure resources, you can build your own or use predefined ones [e.g. for AWS resources](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/aws.py).<br> 


#### Option 1: Using Iguazio data fabric for artifacts
If your are using [Iguazio data science platform](https://www.iguazio.com/) use the `mount_v3io()` modifier,<br>it will attach the function to iguazio real-time data fabric (mount by default to the `Home` of the current user)

> Note: if the notebook is not on the managed platform (running remotely) you need to create and use a v3io secret, run: <br>
`kubectl create -n <namespace> secret generic my-v3io --from-literal=accessKey=<you access key> --from-literal=username=<your user name> --type v3io/fuse`
, <br>and use: `trainer.apply(mount_v3io(user='admin', secret='my-v3io'))`  

In [9]:
# when using Iguazio data science platform
trainer.apply(mount_v3io())
# location of the artifacts
output_path = '/User/mlrun/data'

#### Option 2: Using AWS S3 for artifacts

In AWS you can use S3 and need to have a `secret` with AWS credentials, a secret can be created with the following command line:

`kubectl create -n <namespace> secret generic my-aws --from-literal=AWS_ACCESS_KEY_ID=<access key> --from-literal=AWS_SECRET_ACCESS_KEY=<secret key>`

In [10]:
# when using S3 as the artifact store
from kfp.aws import use_aws_secret
trainer.apply(use_aws_secret(secret_name='my-aws'))
output_path = 's3://<your-bucket-name>/jobs'

### Deploy (build) the function container
the `deploy()` command will build a custom container image (create a cluster build job) from the outlined function dependencies,<br>
we can pass the `image` name instead if we have a pre-build container image. note the code and params can be updated per run without building a new image.

The image is stored in a container repository, by default it is using the one configured on the MLRun API service, you can specify your own docker registry by first creating a secret, and adding that secret name to the build configuration:

`kubectl create -n <namespace> secret docker-registry my-docker --docker-server=https://index.docker.io/v1/ --docker-username=<your-user> --docker-password=<your-password> --docker-email=<your-email>`

and run this: `trainer.build_config(image='target/image:tag', secret='my_docker')`

In [ ]:
# prepare an image from the dependencies, so we wont need to build the image every run 
trainer.deploy(watch=True)

<a id=run-on-cluster></a>
## Run the function on the cluster
note the listfiles call will return the same results as in the local run since the function shares the same filesystem <br>
`with_code()` will inject the latest code to the function, in case we made changes (it doesnt require a new build)

In [14]:
trainer.with_code()

In [11]:
# create the base task (common to both steps)
base_task = NewTask(out_path=output_path).set_label('stage', 'dev')

In [12]:
# run our training task, with hyper params, and select the one with max accuracy
train_task = NewTask(name='my-training', handler='training', params={'p1': 9}, base=base_task)
train_run = trainer.run(train_task, watch=True)

[mlrun] 2019-12-18 21:33:56,491 starting run my-training uid=ceedd24ffde5403d94d0a04e8c2377d1  -> http://mlrun-api:8080
Run: my-trainer (uid=ceedd24ffde5403d94d0a04e8c2377d1)
Params: p1=9, p2=2
[mlrun] 2019-12-18 21:34:06,423 started training

[mlrun] 2019-12-18 21:34:06,448 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...2377d1,0,Dec 18 21:34:06,completed,my-trainer,category=testshost=my-training-fgr62kind=jobowner=adminstage=dev,,p1=9,accuracy=18loss=27,model


to track results use .show() or .logs() or in CLI: 
!mlrun get run ceedd24ffde5403d94d0a04e8c2377d1  , !mlrun logs ceedd24ffde5403d94d0a04e8c2377d1 
[mlrun] 2019-12-18 21:34:15,742 run executed, status=completed


In [14]:
# running validation, use the best model result from the previos step 
model_path = train_run.outputs['model']
trainer.run(base_task, handler='validation', inputs={'model': model_path}, watch=True)

[mlrun] 2019-12-18 21:36:20,746 starting run validation uid=3780847dbc3643819983076a432e0245  -> http://mlrun-api:8080
Run: my-trainer (uid=3780847dbc3643819983076a432e0245)
file - /User/mlrun/data/model.txt:
b'abc is 123'

[mlrun] 2019-12-18 21:36:26,047 started validation

[mlrun] 2019-12-18 21:36:26,068 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...2e0245,0,Dec 18 21:36:26,completed,my-trainer,host=validation-l95wkkind=jobowner=adminstage=dev,model,,,validation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3780847dbc3643819983076a432e0245  , !mlrun logs 3780847dbc3643819983076a432e0245 
[mlrun] 2019-12-18 21:36:29,937 run executed, status=completed


<a id=create-pipeline></a>
## Create and Run a KubeFlow Pipeline
KubeFlow pipelines is used for workflow automation, we for a graph of functions and specify the parameters, inputs and outputs.<br>
output of one step can become the input of the other as ilustrated below

In [15]:
import kfp
from kfp import dsl

# where to store pipeline results (can add "/{{workflow.uid}}" if you want a unique dir per workflow)
artifacts_path = output_path

In [17]:
@dsl.pipeline(
    name='job test',
    description='Show how to use mlrun.'
)
def job_pipeline(
   p1 = 9
):
    task = NewTask( out_path=output_path, outputs=['model']).with_params(p1=p1)
    train = trainer.as_step(handler='training',
                            out_path=artifacts_path, 
                            params={'p1': p1},
                            outputs=['model'])
    
    validate = trainer.as_step(handler='validation',
                               out_path=artifacts_path, 
                               inputs={'model': train.outputs['model']},
                               outputs=['validation'])
    

In [20]:
# used to locally debug kfp dsl
#kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')

In [19]:
client = kfp.Client(namespace='default-tenant')
arguments = {'p1': 8}
run_result = client.create_run_from_pipeline_func(job_pipeline, arguments, experiment_name='my-job')

<b>[GO BACK TO THE TOP](#top)</b>